In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
from nltk.stem.porter import *
from nltk.stem import *
import xgboost as xgb
import pandas as pd
import numpy as np
import spacy.cli
import gensim
import spacy
import nltk

In [3]:
spacy.cli.download("pt_core_news_sm")
nltk.download('wordnet')

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


[nltk_data] Downloading package wordnet to /home/rickecr/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
data = pd.read_csv('textos_videos1.csv')

X = data['texto']
y = data['classes_problema']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [5]:
# Pré-processamento dos dados.

stemmer = PorterStemmer()
nlp = spacy.load('pt_core_news_sm')

# Pré-processamento dos dados.

stemmer = PorterStemmer()
nlp = spacy.load('pt_core_news_sm')

In [6]:
# Lematização
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Colocando todas as palavras para caixa baixa.
def preprocess(text):
  return lemmatize_stemming(text.lower())

In [7]:
class Text2TfIdfTransformer(BaseEstimator):
    def __init__(self):
        self._model = TfidfVectorizer()
        pass

    def fit(self, x, y=None):
        x = x.apply(lambda x : preprocess(x))
        self._model.fit(x)
        return self

    def transform(self, x):
        return self._model.transform(x)

In [12]:
pl_xgb_tf_idf = Pipeline(steps=[('tfidf', Text2TfIdfTransformer()),
                         ('xgboost', xgb.XGBClassifier(objective='multi:softmax'))])

In [13]:
pl_xgb_tf_idf.fit(X_train, y_train)
p = pl_xgb_tf_idf.predict(X_test)

In [14]:
result = np.mean(p == y_test)
print("%.3f" % result)

0.825
